In [18]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
import numpy as np
import ast
headline_vectorizer = CountVectorizer()

In [23]:
news_dataset = pd.read_csv("./data_preprocessing/news_dataset.csv")
news_dataset.drop(['Unnamed: 0'], axis=1, inplace=True)
news_dataset.reset_index(drop=True, inplace=True)

In [25]:
news_dataset.to_csv("news_dataset_clean.csv")

In [26]:
history_dataset = pd.read_csv("./data_preprocessing/click_history.csv")
history_dataset.drop(['Unnamed: 0'], axis = 1, inplace = True)
history_dataset.reset_index(drop=True, inplace=True)
history_dataset['click_history'] = history_dataset['click_history'].fillna('[]')
history_dataset['click_history'] = history_dataset['click_history'].apply(ast.literal_eval)
history_dataset.reset_index(drop=True, inplace=True)

In [27]:
history_dataset.to_csv("history_dataset_clean.csv")

In [6]:
news_dataset = pd.read_csv("./news_data.csv")
history_dataset = pd.read_csv("./history_data.csv")

In [11]:
history_dataset

,Unnamed: 0,userId,click_history
0,0,U13740,"['N55189', 'N42782', 'N34694', 'N45794', 'N184..."
1,1,U91836,"['N31739', 'N6072', 'N63045', 'N23979', 'N3565..."
2,2,U73700,"['N10732', 'N25792', 'N7563', 'N21087', 'N4108..."
3,3,U34670,"['N45729', 'N2203', 'N871', 'N53880', 'N41375'..."
4,4,U8125,"['N10078', 'N56514', 'N14904', 'N33740']"
...,...,...,...
156960,156960,U21593,"['N7432', 'N58559', 'N1954', 'N43353', 'N14343..."
156961,156961,U10123,"['N9803', 'N104', 'N24462', 'N57318', 'N55743'..."
156962,156962,U75630,"['N29898', 'N59704', 'N4408', 'N9803', 'N53644..."
156963,156963,U44625,"['N4118', 'N47297', 'N3164', 'N43295', 'N6056'..."


In [20]:
def euclidean_distance_based_model(title, num_similar_items, data):
    try:
        row_index = data[data['Title'] == title].index[0]
        category = data['Category'][row_index]
        category_data = data[data['Category'] == category]
        headline_features = headline_vectorizer.fit_transform(category_data['Title'].values)
        couple_dist = pairwise_distances(headline_features, headline_features[row_index])
        indices = np.argsort(couple_dist.ravel())[1:num_similar_items + 1] 
        
        similar_titles_dict = {}
        for idx in indices:
            similar_titles_dict[category_data.iloc[idx]['News ID']] = category_data.iloc[idx]['Title']

        return similar_titles_dict
    except:
        return None

user_id = "U44625"
user_click_history = history_dataset[history_dataset.userId == user_id].reset_index()['click_history'][0]

combined_similar_titles = {}  

for history in user_click_history:
    try:
        title = news_dataset[news_dataset['News ID'] == history]['Title'].values[0]
        input_title = title
        similar_titles = euclidean_distance_based_model(input_title, 6, news_dataset)
        if similar_titles is not None:
            combined_similar_titles.update(similar_titles)
    except IndexError:
        pass

print(combined_similar_titles)

{'N33951': 'New ISIS leader named after successful military raid', 'N52183': 'Intelligence, 2 associates captured after ISIS terrorist leader killed', 'N40463': 'Jimmy Carter hospitalized after fall', 'N40893': "Baghdadi's death is not a triumph", 'N36122': 'U.S. Delegation Visits Greenland', 'N22849': "After Math: These aren't deals", 'N60842': 'Bristol announces expansion of two businesses', 'N21002': 'Photos of the Day', 'N29398': 'MTA announces the expansion of OMNY', 'N15680': 'Jeff Sessions announces Senate run', 'N16914': "Who's Afraid of Elizabeth Warren?", 'N29669': 'The Fragility of American Citizenship', 'N5859': "It's voting day! But where to go?", 'N59744': 'NYPD commissioner to resign', 'N41623': "Irvine's 3 best Brazilian steakhouses", 'N29784': "Monday's Connecticut Scholastic Scoreboard", 'N35809': 'Mormons to the Rescue?', 'N20429': 'Veterans Day Parade set for Monday', 'N35406': 'The Best Gifts For Teens', 'N17155': '2 the Rescue: Meet Vertical', 'N21789': 'Look of t